In [1]:
import torch
from pykeops.torch import LazyTensor
from rlaopt.kernels import KernelConfig
from rlaopt.kernels.standard import _kernel_computation_rbf

from experiments.data_processing.load_torch import LOADERS
from experiments.utils import get_gp_hparams

In [2]:
# def _kernel_computation_rbf(
#     Ai_lazy: LazyTensor, Aj_lazy: LazyTensor, kernel_config: KernelConfig
# ):
#     """Compute RBF kernel."""
#     D = _get_scaled_diff(Ai_lazy, Aj_lazy, kernel_config)
#     D = (D**2).sum(dim=2)
#     return _apply_const_scaling((-D / 2).exp(), kernel_config)

In [3]:
seed = 0
torch.manual_seed(seed)

In [4]:
split_proportion = 0.1
split_shuffle = True
standardize = True
dtype = torch.float32
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
load_fn = LOADERS["yolanda"]

In [6]:
dataset = load_fn(
    split_proportion=split_proportion,
    split_shuffle=split_shuffle,
    split_seed=seed,
    standardize=standardize,
    dtype=dtype,
    device=device,
)

In [7]:
gp_hparams = get_gp_hparams("yolanda", "rbf", seed).to(device=device, dtype=dtype)
# kernel_config = KernelConfig(const_scaling=gp_hparams.signal_variance, lengthscale=gp_hparams.kernel_lengthscale)
# kernel_config = KernelConfig(const_scaling=gp_hparams.signal_variance, lengthscale=1.0)
kernel_config = KernelConfig(
    const_scaling=gp_hparams.signal_variance, lengthscale=100 * torch.ones(dataset.Xtr.shape[1], device=device, dtype=dtype))

In [8]:
Xtr_i_lazy = LazyTensor(dataset.Xtr[:, None, :])
Xtr_j_lazy = LazyTensor(dataset.Xtr[None, :, :])
K_lazy = _kernel_computation_rbf(Xtr_i_lazy, Xtr_j_lazy, kernel_config)

kernel config lengthscale shape: torch.Size([100])


In [9]:
w = torch.zeros(K_lazy.shape[1], device=device, dtype=dtype)

In [10]:
mv = K_lazy @ w
print(mv.shape)
print(mv.isnan().any().item())

[KeOps] Generating code for Sum_Reduction reduction (with parameters 0) of formula (d*e)*Exp(-1/2*Sum((b*(a-c))**2)) with a=Var(0,100,0), b=Var(1,100,2), c=Var(2,100,1), d=Var(3,1,2), e=Var(4,1,1) ... OK
torch.Size([360000])
True


In [11]:
print(mv)

tensor([nan, nan, nan,  ..., nan, nan, nan], device='cuda:0')
